In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel values to be between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert the labels to one-hot encoded vectors
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Create a convolutional neural network
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model on the testing data
loss, accuracy = model.evaluate(x_test, y_test)

print(f"Test accuracy: {accuracy}")

Epoch 1/10
1563/1563 [==============================] - 15s 10ms/step - loss: 1.5418 - accuracy: 0.4378 - val_loss: 1.2591 - val_accuracy: 0.5466
Epoch 2/10
1563/1563 [==============================] - 15s 9ms/step - loss: 1.1757 - accuracy: 0.5830 - val_loss: 1.0853 - val_accuracy: 0.6155
Epoch 3/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.0269 - accuracy: 0.6390 - val_loss: 0.9922 - val_accuracy: 0.6537
Epoch 4/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.9359 - accuracy: 0.6735 - val_loss: 0.9864 - val_accuracy: 0.6589
Epoch 5/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.8619 - accuracy: 0.6981 - val_loss: 0.9192 - val_accuracy: 0.6827
Epoch 6/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.8023 - accuracy: 0.7185 - val_loss: 0.9078 - val_accuracy: 0.6831
Epoch 7/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.7553 - accuracy: 0.7358 - val_loss: 0.8845 - 

In [2]:

from numba import jit, cuda
import numpy as np
from timeit import default_timer as timer
import numpy as np

In [11]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [7]:
batch1 = unpickle('cifar-10-batches-py/data_batch_1')
batch2 = unpickle('cifar-10-batches-py/data_batch_2')
batch3 = unpickle('cifar-10-batches-py/data_batch_3')
batch4 = unpickle('cifar-10-batches-py/data_batch_4')
batch5 = unpickle('cifar-10-batches-py/data_batch_5')
batch_meta = unpickle('cifar-10-batches-py/batches.meta')

In [8]:

df1 = pd.DataFrame(batch1[b'data'])
df2 = pd.DataFrame(batch2[b'data'])
df3 = pd.DataFrame(batch3[b'data'])
df4 = pd.DataFrame(batch4[b'data'])
df5 = pd.DataFrame(batch5[b'data']) 

#load the labels into a dataframe
df1_labels = pd.DataFrame(batch1[b'labels'])
df2_labels = pd.DataFrame(batch2[b'labels'])
df3_labels = pd.DataFrame(batch3[b'labels'])
df4_labels = pd.DataFrame(batch4[b'labels'])
df5_labels = pd.DataFrame(batch5[b'labels'])

In [9]:



#concatenate the dataframes
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)
df_labels = pd.concat([df1_labels, df2_labels, df3_labels, df4_labels, df5_labels], ignore_index=True)

#load the data into a tensor

data = torch.from_numpy(df.values)
labels = torch.from_numpy(df_labels.values)

#reshape the data into a 4D tensor
data = data.reshape(50000, 3, 32, 32)



In [12]:
def func(a):								
    for i in range(n):
        a=np.random.randn(n)
        b=np.random.randn(n)
        c=np.dot(a,b)

@jit(target_backend='cuda')
def func2(a):
    for i in range(n):
        A=np.random.randn(n)
        B=np.random.randn(n)
        C=np.dot(A,B)

if __name__=="__main__":
    n=100000
    a=np.random.randn(n)
    # print(a)
    start = timer()
    func(a)
    print("without GPU:", timer()-start)	
	
    start = timer()
    func2(a)
    print("with GPU:", timer()-start)

C:\Users\matan.s\AppData\Local\Temp\ipykernel_23052\3158736793.py:7: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


KeyboardInterrupt: 

In [17]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer

# normal function to run on cpu
def func(a):								
	for i in range(1000000):
		a[i]+= 1	

# function optimized to run on gpu
@jit(target_backend='cuda')						
def func2(a):
	for i in range(1000000):
		a[i]+= 1
if __name__=="__main__":
	n = 1000000		
	a = np.ones(n, dtype = np.float64)
	
	start = timer()
	func(a)
	print("without GPU:", timer()-start)	
	
	start = timer()
	func2(a)
	print("with GPU:", timer()-start)


C:\Users\matan.s\AppData\Local\Temp\ipykernel_23052\4086339893.py:12: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


without GPU: 0.10602420000031998
with GPU: 0.04593100000056438


In [20]:
# gpu_info= !nvidia-smi
gpu_info='\n'.join(gpu_info)
if gpu_info.find('failed')>=0:
    print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

Wed Jun 14 12:53:21 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA T1000 8GB             WDDM  | 00000000:01:00.0  On |                  N/A |
| 35%   42C    P8              N/A /  50W |    917MiB /  8192MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--